In [1]:
# Import Packages:
from __future__ import print_function

import os, sys
from six.moves import cPickle as pickle
from six.moves import range

# Model Packeges import
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds

In [2]:
STDbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/STD/batchData/"
EDGbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/EDG/batchData/"
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp1/batchData/"
HOGp2batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp2/batchData/"
HOGp3batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp3/batchData/"

featureDIR = HOGp3batch_dir

In [3]:
# def toOneHotVector():
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    
def reshape_data(dataset, labels, featureSize, numLabels, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape(sample_size, featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape(len(dataset), featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [4]:
class BuildNeuralNet():
    
    def __init__(self):
        
        seed = 2316
        self.featureSize = 900     # Note to change the featureSize when using a new Feature Set
        self.numHidden1 = 1000
        self.numHidden2 = 1000
        self.numHidden3 = 1000
        self.numLabels = 2
        self.alpha = 0.06
        self.momentum = 0.9
        
        self.weights = {
            "inp_to_hid1_wght": tf.Variable(tf.random_normal([self.featureSize, self.numHidden1], seed=seed)),
            "hid1_to_hid2_wght" : tf.Variable(tf.random_normal([self.numHidden1, self.numHidden2], seed=seed)),
            "hid2_to_hid3_wght": tf.Variable(tf.random_normal([self.numHidden2, self.numHidden3], seed=seed)),
            "hid3_to_out_wght": tf.Variable(tf.random_normal([self.numHidden3, self.numLabels], seed=seed))
        }
        
        self.biases = {
            "hid1_bias": tf.Variable(tf.random_normal([self.numHidden1], seed=seed)),
            "hid2_bias": tf.Variable(tf.random_normal([self.numHidden2], seed=seed)),
            "hid3_bias": tf.Variable(tf.random_normal([self.numHidden3], seed=seed)),
            "out_bias": tf.Variable(tf.random_normal([self.numLabels], seed=seed))
        }
        
        
    def trainNet(self):
        trainData = tf.placeholder(tf.float32, [None, self.featureSize])
        trainLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # 1st Hidden Layer State
        inp_to_hid1 = tf.matmul(trainData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias']
        hid1State = tf.sigmoid(inp_to_hid1)
        
        # 2nd Hidden Layer State
        hid1_to_hid2 = tf.matmul(hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias']
        hid2State = tf.sigmoid(hid1_to_hid2)
        
        # 3nd Hidden Layer State
        hid2_to_hid3 = tf.matmul(hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias']
        hid3State = tf.sigmoid(hid2_to_hid3)
        
        # Output Layer State
        hid3_to_out = tf.matmul(hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias']
        outState = tf.nn.softmax(hid3_to_out)
        
        # Loss Function and Optimization
        lossCE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hid3_to_out, labels=trainLabels))
#         optimizer = tf.train.AdamOptimizer(learning_rate=self.alpha).minimize(lossCE)
        optimizer = tf.train.MomentumOptimizer(self.alpha, 
                                            self.momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(lossCE)
        
        # Evaluate model
#         trainPred = tf.equal(tf.argmax(outState, 1), tf.argmax(trainLabels, 1))
#         trainAccuracy = tf.reduce_mean(tf.cast(trainPred, tf.float32))
        
        return dict(
            trainData = trainData, 
            trainLabels = trainLabels,
#             weightsLRND = self.weights,
            trainPred = outState,
            optimizer = optimizer,
            lossCE = lossCE
        )
    
    
    def crossValid(self):
            validData = tf.placeholder(tf.float32, [None, self.featureSize])
            validLabels = tf.placeholder(tf.float32, [None, self.numLabels])

            # 1st Hidden Layer State
            valid_hid1State = tf.sigmoid(tf.matmul(validData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias'])

            # 2nd Hidden Layer State
            valid_hid2State = tf.sigmoid(tf.matmul(valid_hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias'])

            # 3nd Hidden Layer State
            valid_hid3State = tf.sigmoid(tf.matmul(valid_hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias'])

            # Output Layer State
            validOutState = tf.nn.softmax(tf.matmul(valid_hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias'])

#             validPred = tf.equal(tf.argmax(validOutState, 1), tf.argmax(validLabels, 1))
#             validAccuracy = tf.reduce_mean(tf.cast(validPred, tf.float32))
            
            return dict(
                validData = validData,
                validLabels = validLabels,
                validPred = validOutState
            )
            
            

In [5]:
def accuracy(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

def confusionMatrix(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (pd.crosstab(np.argmax(labels, 1), np.argmax(predictions, 1), rownames=['True'], colnames=['Predicted'], margins=True))


class SessionExec():
    
    def __init__(self):
        self.epochs = 30
        
    def model(self, trainDataIN, trainLabelsIN, validDataIN, validLabelsIN):
#         print (self.trainGraphDict)
#         print ('')
#         print (self.validGraphDict)

        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())

            for epoch in range(self.epochs): 
                feed_dict = {self.trainGraphDict['trainData']: trainDataIN,
                             self.trainGraphDict['trainLabels']: trainLabelsIN
                        }


                _, loss, tpred = sess.run([self.trainGraphDict['optimizer'], 
                                                        self.trainGraphDict['lossCE'],
                                                        self.trainGraphDict['trainPred']], feed_dict=feed_dict)


                if (epoch+1)%2 == 0:
                    # Evaluate training set
                    tacc = accuracy(tpred, trainLabelsIN)
                    print ("Fold: " + str(self.foldNUM+1) + ", Iter: " + str(epoch+1) + ", Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(tacc))

                    
            # After all epoch are over with, test the model with the cross validation set
            feed_dict = {self.validGraphDict['validData']: validDataIN,
                     self.validGraphDict['validLabels']: validLabelsIN
                }

            vpred = sess.run(self.validGraphDict['validPred'], feed_dict=feed_dict)
            
            # Evaluate corss-validation set
            vacc = accuracy(vpred, validLabelsIN)
            print ("Fold: " + str(self.foldNUM+1) + ", Cross Validation Accuracy= " + "{:.5f}".format(vacc))

            print ('')
            print ('')
            
            return tpred, tacc, vpred, vacc 



    def execute(self):
        meanValidAcc = 0
        for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(featureDIR, oneHot=True)):
            print ('')
            print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print ('The Label Dictionary is given as: ', labelDict)
            self.foldNUM = foldNUM
            
            reset_graph()
            objNNET = BuildNeuralNet()
            self.trainGraphDict = objNNET.trainNet()
            self.validGraphDict = objNNET.crossValid()
            tpred, tacc, vpred, vacc = self.model(trainDataIN, trainLabelsIN, validDataIN, validLabelsIN)
            
            trainCM = confusionMatrix(trainLabelsIN,tpred)
            validCM = confusionMatrix(validLabelsIN,vpred)

            meanValidAcc += vacc
            print ('Confusion Matrix Training Set')
            print (trainCM)
            print ('')
            print ('Confusion Matrix CrossValid Set')
            print (validCM) 
#             break
        print ('')
        print ('The Mean crossValidation Accuracy is: ', meanValidAcc/(foldNUM+1))
            
            
SessionExec().execute()




# Running i is : 0
# Validation Data and Labels shape:  (1000, 162) (1000, 2)
# Training Data and Labels shape:  (9000, 162) (9000, 2)
# The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
# Fold: 1, Iter: 10, Loss= 1.603019, Training Accuracy= 0.79000
# Fold: 1, Cross Validation Accuracy= 0.79200


Validation Data and Labels shape:  (1000, 900) (1000, 2)
Training Data and Labels shape:  (9000, 900) (9000, 2)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
Fold: 1, Iter: 2, Loss= 16.883253, Training Accuracy= 50.08889
Fold: 1, Iter: 4, Loss= 14.522053, Training Accuracy= 50.45556
Fold: 1, Iter: 6, Loss= 10.628699, Training Accuracy= 51.95556
Fold: 1, Iter: 8, Loss= 7.603653, Training Accuracy= 56.34444
Fold: 1, Iter: 10, Loss= 5.946406, Training Accuracy= 61.11111
Fold: 1, Iter: 12, Loss= 3.611921, Training Accuracy= 70.77778
Fold: 1, Iter: 14, Loss= 2.456172, Training Accuracy= 77.38889
Fold: 1, Iter: 16, Loss= 2.070004, Training Accuracy= 80.00000
Fold: 1, Iter: 18, Loss= 1.889840, Training Accuracy= 81.32222
Fold: 1, Iter: 20, Loss= 1.709457, Training Accuracy= 82.63333
Fold: 1, Iter: 22, Loss= 1.530144, Training Accuracy= 83.58889
Fold: 1, Iter: 24, Loss= 1.363107, Training Accuracy= 84.62222
Fold: 1, Iter: 26, Loss= 1.210799, Trai

In [6]:
# def accuracy(predictions, labels):
#     return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
#           / predictions.shape[0])


# epochs = 30
# def model(trainGraphDict, validGraphDict):
#     print (trainGraphDict)
#     print ('')
#     print (validGraphDict)
    
#     with tf.Session() as sess:
#         sess.run(tf.initialize_all_variables())
        
#         for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(featureDIR, oneHot=True)):
#             print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
#             print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
#             print ('The Label Dictionary is given as: ', labelDict)

#             for epoch in range(epochs): 
#                 feed_dict = {trainGraphDict['trainData']: trainDataIN,
#                              trainGraphDict['trainLabels']: trainLabelsIN
#                         }
                
                
#                 _, loss, tacc, twghts, tbiases = sess.run([trainGraphDict['optimizer'], 
#                                                         trainGraphDict['lossCE'], 
#                                                         trainGraphDict['accuracy'],
#                                                         trainGraphDict['weightsLRND'],
#                                                         trainGraphDict['biasesLRND']], feed_dict=feed_dict)

#                 if (epoch+1)%10 == 0:
#                     print ("Fold: " + str(foldNUM+1) + ", Iter: " + str(epoch) + ", Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(tacc))

#                 # During the Last iteration of the nth fold batch, we should calculate the model accuracy with Cross Validataion Data
#                 if epoch==epochs-1:
#                     print ('')

#                     feed_dict = {validGraphDict['validData']: validDataIN,
#                              validGraphDict['validLabels']: validLabelsIN
#                         }
                
#                     vacc = sess.run(validGraphDict['validAccuracy'], feed_dict=feed_dict)
#                     print ("Fold: " + str(foldNUM+1) + ", Cross Validation Accuracy= " + "{:.5f}".format(vacc))
                    
#             print ('')
#             print ('')

# #             break
# #             epoch += 1
# #         print ("Optimization Finished!")
# #     obj_SVM = Models()
# #     batchEvalDict[foldNUM] = obj_SVM.classify(trainData, trainLabels, validData)

# reset_graph()
# objNNET = BuildNeuralNet()
# trainGraphDict = objNNET.trainNet()
# validGraphDict = objNNET.crossValid()
# model(trainGraphDict, validGraphDict)

In [7]:
c = []
a = [1,2,3,4]
b = [4,5,6,7]

c.append(a+b)
# c.append(b)
c

(a,b,c,d) = (1,2,3,4)
c

3